# Python Camp 2018: Einführung GeoPandas

### Github: https://github.com/martinchristen/pythoncamp2018


## Dokumentation

http://geopandas.readthedocs.io


## Installation

    # Eventuell Python 3.5 verwenden, immer noch bessere Geo-Unterstützung (z.B. GDAL)
    conda create -n py35 python=3.5 ipykernel
    source activate py35    

    conda install geopandas 
    conda install -c conda-forge folium


## Datenquellen für das heutige Tutorial

* https://www.bfs.admin.ch/bfs/de/home/dienstleistungen/geostat/geodaten-bundesstatistik.html
* https://earthquake.usgs.gov/earthquakes/feed/v1.0/geojson.php
* http://www.geonames.org/

In [ ]:
import geopandas as gpd

Laden eines ESRI Shapefiles, andere Formate, z.B. GeoJSON werden auch unterstützt. Die vollständige Liste unterstützter Vektor-Formate erfolgt über das fiona Modul:

    import fiona
    fiona.supported_drivers

In [ ]:
import fiona
fiona.supported_drivers

(Für Vektorformate siehe z.B. auch: http://www.gdal.org/ogr_formats.html )

### GeoDataFrame aus einem CSV erstellen

Dazu verwenden wir den Datensatz von GeoNames (city5000, Weltweite Städte mit Population > 5000)

In [ ]:
from shapely.geometry import Point
import pandas as pd

In [ ]:
df = pd.read_csv('daten/cities5000.txt', sep="\t", header=None, low_memory=False)

In [ ]:
df.head()

In [ ]:
df2 = df[[1,4,5,14,18]]
df2.head()

In [ ]:
df2.columns = ["name", "lat", "lng", "population", "datum"]

In [ ]:
df2.head()

In [ ]:
df2.query("name == 'Köln'")

Erstellen eines **GeoDataFrame**

In [ ]:
geometry = [Point(pos) for pos in zip(df2['lng'], df2['lat'])]
gdf = gpd.GeoDataFrame(df2, geometry=geometry)

In [ ]:
gdf.head()

dies ist nun ein GeoDataFrame, also eigentlich ein DataFrame (pandas) mit einer Spalte "geometry".

Die Dokumentation zu geopandas ist auf: http://geopandas.readthedocs.io


Wir können das Resultat beispielsweise als Shapefile speichern:

In [ ]:
gdf.to_file("cities.shp", driver="Shapefile", encoding="utf-8")

## Kantone

Das folgende Shapefile ist aus dem Datensatz "Generalisierte Gemeindegrenzen" vom Bundesamt für Statistik. Aus Performance-Gründen verwenden wir heute generalisierte Daten. Der detailliertere Datensatz wäre natürlich **swissBOUNDARIES3D** von swisstopo, welcher auch kostenlos bezogen werden kann. ( https://shop.swisstopo.admin.ch/de/products/landscape/boundaries3D )

In [ ]:
kantone = gpd.read_file("daten/kantone.shp", encoding="utf-8")

## Überblick des Datensatzes

Sehen wir uns die ersten 5 Einträge an. Dies geschieht mit der Methode .head()

In [ ]:
kantone.head()

In [ ]:
kantone.columns

* Die **Beschreibung** der Attribute ist im PDF des Datensatzes zu finden.

* In diesem Datensatz haben wir sehr viele Sachdaten, welche auch eine Geometrie beschreiben. Das ist eigenlich unschön, sehen wir uns das später an.

In [ ]:
kantone.shape

In [ ]:
kantone

In [ ]:
%matplotlib inline
kantone.plot(figsize=(15,9));

In [ ]:
kantone.crs

Datensatz nach WGS84 konvertieren

(Word Geodetic System 1984 - ein globales Referenzsystem der Geodäsie und Navigation)

In [ ]:
kantoneWGS84 = kantone.to_crs(epsg=4326)
kantoneWGS84.plot(figsize=(15,9));

# Darstellung in Folium

(-> Folium Tutorial eventuell am Sonntag am PythonCamp )

In [ ]:
import folium

center = [47.534018, 7.638423]
karte = folium.Map(center, zoom_start=6, tiles='cartodbpositron')   
# weitere Karten siehe: https://deparkes.co.uk/2016/06/10/folium-map-tiles/

folium.GeoJson(kantoneWGS84).add_to(karte)

karte

In [ ]:
import folium

def my_color_function(feature):
    area = feature["properties"]["AREA_HA"]
    
    color = "#000000"
    
    if area >10000:
        color = "#110000"
    if area >40000:
        color = "#440000"
    if area >70000:
        color = "#770000"
    if area >100000:
        color = "#AA0000"
    if area >130000:
        color = "#DD0000"
    if area >160000:
        color = "#FF0000"
    
    return color

center = [47.534018, 7.638423]
karte = folium.Map(center, zoom_start=6, tiles='cartodbpositron')   
# weitere Karten siehe: https://deparkes.co.uk/2016/06/10/folium-map-tiles/

folium.GeoJson(kantoneWGS84,style_function=lambda feature: {
        'fillColor': my_color_function(feature),
        'color': 'black',
        'weight': 2,
        'dashArray': '5, 5'
    }).add_to(karte)

karte

In [ ]:
data = []

for i in range(0,len(kantoneWGS84)):
    data.append({'fillColor': 'rgb(255,255,0)', 'weight': 2, 'color': 'black'})
    
kantoneWGS84['style'] = data
kantoneWGS84


In [ ]:
kantoneWGS84.columns

In [ ]:
import folium

center = [47.534018, 7.638423]
karte = folium.Map(center, zoom_start=6, tiles='cartodbpositron')   
# weitere Karten siehe: https://deparkes.co.uk/2016/06/10/folium-map-tiles/

folium.GeoJson(kantoneWGS84).add_to(karte)
karte

## Abfragen

Anlegen eines neuen GeoDataFrames mit reduzierter Anzahl von Attributen

In [ ]:
kt = kantone[["KTNR","KTNAME", "AREA_HA", "geometry"]]

In [ ]:
kt.head()

Der Datensatz kann nun gespeichert werden, z.B. als ERSI Shapefile

In [ ]:
kt.to_file("kantone_reduziert.shp", driver="Shapefile", encoding="utf-8")

In [ ]:
kt2 = gpd.read_file("kantone_reduziert.shp", encoding="utf-8")
kt2.head()

In [ ]:
del kt2

GeoPandas enthält zahlreiche Funktionalität, aus Zeitgründen werden wir heute nur ein paar davon ansehen.

Wir sortieren z.B. die Daten nach der Fläche. Weitere Operationen sehen wir mit einem anderen Datensatz an.

In [ ]:
kt.sort_values(['AREA_HA'], ascending=False)

### Räumliche Abfragen / Operationen

In [ ]:
from shapely.geometry import Point

FHNW = Point(2615044, 1264828)  # Gründenstrasse 40, 4132 Muttenz (LV95)

In [ ]:
kt.contains(FHNW)

In [ ]:
kt[kt.contains(FHNW)]

In [ ]:
dist = kt.distance(FHNW) / 1000
dist

In [ ]:
kt = kantone[["KTNR","KTNAME", "AREA_HA", "geometry"]].copy()
kt["distance"] = dist
kt

In [ ]:
kt.sort_values(["distance"], ascending=True)

## Visualisierung der Daten (Pandas...)

Bar plots, Histograms, Box Plots, Area Plot, Scatter Plot, Hexagonal Bin Plot, Pie plot

Siehe auch: https://pandas.pydata.org/pandas-docs/stable/visualization.html

Das Thema heute ist jedoch nicht die Visualisierung der Daten, dies ist nur zur Info...

In [ ]:
kt[["AREA_HA"]].hist();

In [ ]:
kt[["AREA_HA"]].hist(bins=20);

## Nächster Datensatz: Gemeinden

Sehen wir uns einen weiteren, Datensatz von BFS an: Aufteilung in Gemeinden

In [ ]:
gemeinden = gpd.read_file("daten/gemeinden.shp", encoding="utf-8")

In [ ]:
gemeinden.shape

In [ ]:
gemeinden.columns

In [ ]:
gem = gemeinden[["GMDNAME", "GMDNR", "KTNR", "AREA_HA", "geometry"]]
gem.head()

In [ ]:
gem.plot(figsize=(15,9))

In [ ]:
for i in range(0,len(gem)):
    gs = gem.iloc[i]
    if gs.AREA_HA > 20000:
        print(gs.GMDNAME, gs.AREA_HA)

In [ ]:
gem[gem.AREA_HA > 20000]

In [ ]:
gem[gem.AREA_HA > 20000].plot()

besser: Mehrfachplot...

In [ ]:
kantonsplot = kantone.plot(figsize=(15,9), color="gray")
gem[gem.AREA_HA > 20000].plot(ax=kantonsplot)

In [ ]:
gem.query("AREA_HA > 20000 and AREA_HA < 30000")

In [ ]:
gem.query("GMDNAME == 'Zürich'")

In [ ]:
gem.query("GMDNAME == 'Muttenz'")

In [ ]:
gem.query("GMDNAME == 'Muttenz' or GMDNAME == 'Pratteln'")

In [ ]:
gem.query("GMDNAME == 'Muttenz' or GMDNAME == 'Pratteln'").plot()

In [ ]:
gem.query("GMDNAME == 'Basel'").plot(figsize=(16,8), color="orange")

In [ ]:
kantonsplot = kantone.plot(figsize=(15,9), color="gray")

basel = gem.query("GMDNAME == 'Basel'")
zurich = gem.query("GMDNAME == 'Zürich'")

basel.plot(ax=kantonsplot, color="orange")
zurich.plot(ax=kantonsplot, color="blue")


In [ ]:
kantonsplot = kantone.plot(figsize=(15,9), color="gray")

basel = gem.query("GMDNAME == 'Basel'")
allzurich = gem.query("KTNR == 1")

basel.plot(ax=kantonsplot, color="orange", alpha=0.5)
allzurich.plot(ax=kantonsplot, color="blue", alpha=0.25)


## Live Daten vom Internet: Erdbeben

https://earthquake.usgs.gov/earthquakes/feed/v1.0/geojson.php

In [ ]:
import requests

url = "https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/2.5_week.geojson"
#url = "https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/significant_month.geojson"
#url = "https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/2.5_month.geojson"

data = requests.get(url)
file = open("earthquakes.geojson","wb")
file.write(data.content)
file.close()

für die Beschreibung der Attribute:

https://earthquake.usgs.gov/data/comcat/data-eventterms.php

In [ ]:
erdbeben = gpd.read_file("earthquakes.geojson")
erdbeben.head()

In [ ]:
erdbeben.shape

In [ ]:
erdbeben.columns

In [ ]:
eb = erdbeben[["time","mag", "place","geometry"]].copy()
eb.head()

In [ ]:
eb.mag.hist(bins=32);

In [ ]:
from datetime import datetime, timezone

data = []
for zeile in range(0,len(eb)):
    time = eb.iloc[zeile].time
    t = str(datetime.fromtimestamp(time/1000.0, timezone.utc))
    data.append(t)
    
eb["time_utc"] = data
eb.head()

### Nach Magnitude (mag) sortieren

In [ ]:
eb.sort_values(["mag"], ascending=False).head()

In [ ]:
eb.plot()

In [ ]:
gdfContinents=gpd.read_file("daten/continent.shp", encoding="utf-8")
gdfContinents.head()

In [ ]:
continents = gdfContinents.plot(figsize=(15,9), color="black")
eb.plot(ax=continents, color="red", markersize=10);

## Hinweise

Die Shapefiles vom BFS sind scheinbar nicht korrekt kodiert und wurden folgendermassen bereinigt (Beispiel Kantone):

    kantone = gpd.read_file("data/kantone.shp", encoding="utf-8")  
    kopie_kantone = kantone.iloc[0:0]   # Spalten behalten

    data = []
    for i, j in kantone.iterrows():
        j[1] = j[1].encode("cp1252").decode("utf-8")
        data.append(j)

    kopie_kantone = kopie_kantone.append(data)
    kopie_kantone

    kopie_kantone.to_file("data/kantone2.shp", driver="Shapefile", encoding="utf-8")


### GeoPandas und PostGIS

    import geopandas as gpd
    import psycopg2

    con = psycopg2.connect(database="db", user="user",password="pw",host="host")

    sql= "select geom, x,y,z from your_table"

    df=gpd.GeoDataFrame.from_postgis(sql,con,geom_col='geom')

# Wie weiter ?


z.B. http://2018.geopython.net/#schedule
-> z.B. 5 Stunden Workshops zu GeoPandas / Spatial Data Science mit GeoPandas

